In [1]:
import pandas as pd
import re

In [51]:
import mysql.connector
db = mysql.connector.connect(user='root', password='password', database='folklore_db2')
cur = db.cursor()

In [3]:
from collections import defaultdict

In [52]:
cur.execute("SELECT word, id FROM keywords")
keywords = {x.lower().strip('.'): y for x, y in cur.fetchall()}

In [ ]:
# from numpy import nan
# from collections import namedtuple
# import time

# from pymystem3 import Mystem
# m = Mystem()

In [5]:
table = pd.read_excel('./2020_file/Таблица_архив.xlsx', sheet_name=None)

## Проверка

In [6]:
table.keys()

dict_keys(['Лист 1', 'informators', 'collectors', 'Настя', 'Аня', 'Настя См.', 'Сюзанна', 'ошибки в ключевых словах'])

In [7]:
real = ["Настя", "Настя См.", "Сюзанна"]

In [8]:
df = pd.concat([table[who] for who in real])

In [9]:
df.head(3)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,ссылка на видео,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели,год рождения,место рождения,место проживания,Unnamed: 15,Unnamed: 16
0,1-2019-1,2019,Легенда,"поговорка, храм, колокол, Бог",https://drive.google.com/drive/u/6/folders/1be...,https://drive.google.com/drive/u/6/folders/1be...,-,12:17,17:40,Ивановская область,Южский,Хотимль,NaN,Кулакова Лидия Николаевна \nКулаков Владимир Н...,Заикина А.\nПинаева М.\nКопаева А.,1951; 1943,Домнино; Емельяново,Хотимль,NaN,NaN
1,1-2019-2,2019,Предание топонимическое,предания топоним.,https://drive.google.com/drive/u/6/folders/1be...,https://drive.google.com/drive/u/6/folders/1be...,-,20:00,20:45,Ивановская область,Южский,Хотимль,NaN,Кулакова Лидия Николаевна \nКулаков Владимир Н...,Заикина А.\nПинаева М.\nКопаева А.,1951; 1943,Домнино; Емельяново,Хотимль,NaN,NaN
2,1-2019-3,2019,Народная кулинария,"хлеб, печенье фигурное, календарь, сорок мучен...",https://drive.google.com/drive/u/6/folders/1be...,https://drive.google.com/drive/u/6/folders/1be...,-,25:32,31:23,Ивановская область,Южский,Хотимль,NaN,Кулакова Лидия Николаевна \nКулаков Владимир Н...,Заикина А.\nПинаева М.\nКопаева А.,1951; 1943,Домнино; Емельяново,Хотимль,NaN,NaN


**ключевые слова**

In [17]:
real = ["Настя", "Настя См.", "Сюзанна"]
counter = defaultdict(int)
for words in df['ключевые слова'].values:
    if type(words) == str:
        words = words.lower()
        if 'яйцо пасх. время' in words:
            words = words.replace('яйцо пасх. время', 'яйцо пасх., время')
        if ',' in words:
            sep = ','
        else:
            sep = '.'
        for word in words.strip().split(sep):
            if word:
                counter[word.strip('. \n\t').lower()] += 1

In [19]:
# for key in sorted(counter, key=counter.get, reverse=True):
#     if key not in keywords:
#         print(key)

**информанты**

In [20]:
counter = defaultdict(int)
infs = set()
for inform in df['информанты'].values:
    if ',' in inform:
        sep = ','
    elif ';' in inform:
        sep = ';'
    elif '\n' in inform:
        sep = '\n'
    else:
        sep = None
    if sep:
        for i in inform.split(sep):
            infs.add(i.strip())
    else:
        infs.add(inform)

In [ ]:
for i in sorted(infs):
    print(i)

In [66]:
splitting = {
    'Вера Павловна, Аня Копаева, Саша Гусева ': ['Вера Павловна', 'Аня Копаева', 'Гусева Александра'],
    'Заикина А.\nПинаева М.\nКопаева А.': ['Заикина А.', 'Пинаева М.', 'Копаева А.'],
    'Заикина А. \nСмирнова А.': ['Заикина А.', 'Смирнова А.'],
    'Заикина А. Смирнова А.': ['Заикина А.', 'Смирнова А.'],
    'Заикина Анастасия\nСмирнова Анастасия\nГусева Александра\n': [
        'Заикина Анастасия', 'Смирнова Анастасия', 'Гусева Александра'],
    'Заикина Н., Сельченкова С.': ['Заикина Н.', 'Сельченкова С.'],
    'Кувшинская Ю. М., Демина Д.': ['Кувшинская Ю. М.', 'Демина Д.'],
    'Кувшинская Ю., Сельченкова С.': ['Кувшинская Ю.', 'Сельченкова С.'],
    'Кувшинская Ю., Фесенко В., Сельченкова С.': ['Кувшинская Ю.', 'Фесенко В.', 'Сельченкова С.'],
    'Кувшинская Ю.М., Ходневич Т.': ['Кувшинская Ю.М.', 'Ходневич Т.'],
    'Кувшинская Ю.М., Ходневич Т., Яськова А., Белова М.':[
        'Кувшинская Ю.М.', 'Ходневич Т.', 'Яськова А.', 'Белова М.'
    ],
    'Смирнова А. Фесенко В.': ['Смирнова А.', 'Фесенко В.'],
    'Фесенко В., Сельченкова С.': ['Фесенко В.', 'Сельченкова С.'],
    'Фесенко В., Сельченкова С., Копаева А.': ['Фесенко В.', 'Сельченкова С.', 'Копаева А.'],
    'Ходневич Т. С., Заикина А. А.': ['Ходневич Т. С.', 'Заикина А. А.'],
    'Ходневич Т. С., Яськова А. А., Белова М. В.': ['Ходневич Т. С.', 'Яськова А. А.', 'Белова М. В.'],
    "Фесенко Вера Павловна": ["Фесенко Вера Павловна"]
}

In [26]:
counter = defaultdict(int)
infs = set()
for inform in df['собиратели'].values:
    if type(inform) != str:
        pass
    else:
        infs.update(splitting[inform])

In [27]:
collector_map = {
    'Аня Копаева': 40,
    'Белова М.': 53,
    'Белова М. В.': 53,
    'Вера Павловна': 46,
    'Гусева Александра': 59,
    'Демина Д.': 35,
    'Заикина А.': 36,
    'Заикина А. А.': 36,
    'Заикина Анастасия': 36,
    'Заикина Н.': 36,
    'Копаева А.': 40,
    'Кувшинская Ю.': 49,
    'Кувшинская Ю. М.': 49,
    'Кувшинская Ю.М.': 49,
    'Пинаева М.': 43,
    'Сельченкова С.': 60,
    'Смирнова А.': 61,
    'Смирнова Анастасия': 61,
    'Фесенко В.': 46,
    'Фесенко Вера Павловна': 46,
    'Ходневич Т.': 47,
    'Ходневич Т. С.': 47,
    'Яськова А.': 52,
    'Яськова А. А.': 52
}

**жанры**

In [28]:
cur.execute("SELECT genre_name, id FROM genres;")
genres = dict(cur.fetchall())

In [69]:
genre_mapping = {
    "Порча": "Порча (вредоносная магия и избавление от нее)", 
    "Обряд покупки  скота и ухода за скотом": "Обряд покупки скота и ухода за скотом", 
    "Обряды покупки скота и ухода за скотом": "Обряд покупки скота и ухода за скотом", 
    "Фольклорные коммуникативные формулы": "Этикет", "Жестокий романс": "Жестокий романс, баллада", 
    "Хозяйственный обряд": "Обряд покупки скота и ухода за скотом", "Диалект": "Народная ономастика", 
    "Топонимическое предание": "Предание топонимическое", "Рассказ о гулянии": "Рассказ о гуляньях", 
    "Игры": "Игра", "Оберег, молитва": "Оберег", "Частушка": "Частушки"}

In [30]:
vals = defaultdict(int)
for val in df['жанр'].values:
    val = val.strip()
    vals[val] += 1

for key in sorted(vals, key=vals.get, reverse=True):
    if key not in genres:
        print(key)

Порча
Обряд покупки  скота и ухода за скотом
Обряды покупки скота и ухода за скотом
Фольклорные коммуникативные формулы
Жестокий романс
Хозяйственный обряд
Диалект
Топонимическое предание
Рассказ о гулянии
Игры
Оберег, молитва
Частушка


**тексты**

In [1]:
# for i in df['id'].values:
#     try:
#         print(open(f'./2020_file/txt/{i}.txt').read()[:10])
#     except:
#         print(i)

## Загрузка

**1.Тексты**

In [34]:
import re

In [64]:
def get_text(text):
    text = re.split("Ключевые слова:.*?\n", text)[-1]
    text = re.sub("(^|\n)([СC]:.*?)\n", "\n[\g<2>]\n", text)
    return text.strip()

In [54]:
data = []
for i in df['id'].values:
    text = open(f'./2020_file/txt/{i}.txt').read()
    data.append((i, get_text(text)))

In [55]:
cur.executemany("INSERT INTO texts (old_id, raw_text) VALUES (%s, %s)", data)
db.commit()

In [46]:
cur.execute("UPDATE texts SET leader = 'ЮМК' WHERE id > 6913")
db.commit()

In [48]:
cur.execute("UPDATE texts SET year = 2018 WHERE id > 6913 AND old_id LIKE '%2018%'")
db.commit()

In [65]:
data = []
for i in df['id'].values:
    text = open(f'./2020_file/txt/{i}.txt').read()
    data.append((get_text(text), i,))

In [24]:
# print(data[0][0])

In [67]:
cur.executemany("UPDATE texts SET raw_text = %s WHERE old_id = %s", data)
db.commit()

**2. Жанры**

In [70]:
cur.execute("SELECT genre_name, id FROM genres;")
genres = dict(cur.fetchall())

In [71]:
genre_mapping = {
    "Порча": "Порча (вредоносная магия и избавление от нее)", 
    "Обряд покупки  скота и ухода за скотом": "Обряд покупки скота и ухода за скотом", 
    "Обряды покупки скота и ухода за скотом": "Обряд покупки скота и ухода за скотом", 
    "Фольклорные коммуникативные формулы": "Этикет", "Жестокий романс": "Жестокий романс, баллада", 
    "Хозяйственный обряд": "Обряд покупки скота и ухода за скотом", "Диалект": "Народная ономастика", 
    "Топонимическое предание": "Предание топонимическое", "Рассказ о гулянии": "Рассказ о гуляньях", 
    "Игры": "Игра", "Оберег, молитва": "Оберег", "Частушка": "Частушки"}

In [74]:
df.head(1)

,id,год,жанр,ключевые слова,ссылка на текст,ссылка на аудио,ссылка на видео,время начала,время конца,регион,район,деревня,адрес,информанты,собиратели,год рождения,место рождения,место проживания,Unnamed: 15,Unnamed: 16
0,1-2019-1,2019,Легенда,"поговорка, храм, колокол, Бог",https://drive.google.com/drive/u/6/folders/1be...,https://drive.google.com/drive/u/6/folders/1be...,-,12:17,17:40,Ивановская область,Южский,Хотимль,NaN,Кулакова Лидия Николаевна \nКулаков Владимир Н...,Заикина А.\nПинаева М.\nКопаева А.,1951; 1943,Домнино; Емельяново,Хотимль,NaN,NaN


In [64]:
# genres

In [86]:
data = []
for idx, genre in df[['id','жанр']].values:
    genre = genre.strip()
    if genre in genres:
        g = genre
    else:
#         print(genre, genre_mapping.get(genre))
        g = genre_mapping.get(genre)
    data.append((g, idx))

In [88]:
cur.executemany("UPDATE texts SET genre = %s WHERE old_id = %s", data)
db.commit()

**3.Гео**

In [19]:
geo_mapping = {
    "Хотимль": 73, # 43
    "Волокобино": 74, # 44
    "Емельяново": 75, # 45
    "Преображенское": 76, # 46
    "Ряполово": 77, # 47
    "Нижний Ландех": 12, # 12
    "Лукино": 17 # 17
}

In [20]:
data = []
for idx, vil in df[['id','деревня']].values:
    vil = vil.strip()
    if vil in geo_mapping:
        g = geo_mapping[vil]
    else:
        print(vil)
    data.append((g, idx))

In [21]:
cur.execute("SELECT id FROM g_geo_text")
geo_ids = set([i[0] for i in cur.fetchall()])

In [22]:
for k in geo_mapping.values():
    if k not in geo_ids:
        print(k)

In [23]:
cur.executemany("UPDATE texts SET geo_id = %s WHERE old_id = %s", data)
db.commit()

**4.Ключевые слова**

In [53]:
cur.execute("SELECT old_id, id FROM texts WHERE id > 6913")
texts = dict(cur.fetchall())

In [54]:
cur.execute("SELECT word, id FROM keywords")
keywords = {x.lower().strip('.'): y for x, y in cur.fetchall()}

In [57]:
keyword_mapping = {
    "берёза": "Береза", "болезни": "Болезнь", "быличка": "Былички", "вербное воскресенье": "Вербное воскресение",
    "возмездие": "Кара", "гадание": "Гадания", "драки": "Драка", "заговор": "Заговоры",
    "застолье": "Трапеза ритуальная", "игра фольк": "Игры фольк.", "игры": "Игры фольк.", "костёр": "Костер",
    "красота (свадеб.)": "Красота (свад.)", "крещение": "Крещенье (праздник)", "крещенье": "Крещенье (праздник)",
    "легенда": "Легенды", "молодожёны": "Молодожены", "муж": "Супруги", "наказание": "Кара",
    "народная медицина": "Медицина", "народы": "Народ", "нога": "Ноги", "оберег": "Обереги",
    "обмывание": "Обмывание покойника", "окропить": "Кропить", "песня": "Песни", "поверье": "Демонология",
    "покров": "Покров (календ.)", "поминальный дни": "Поминальные дни", "похороны": "Похоронный обряд",
    "предание": "Предания", "предание топоним": "Предания топоним.", "преображение": "Преображение (календ.)",
    "престольный праздник": "Престольный праздник", "приговор": "Приговоры", "примета": "Приметы",
    "ребёнок": "Ребенок", "рекрутский обряд": "Рекрутские обряды", "ритуальная речь": "Речь ритуальная",
    "свадебный поезд": "Поезд свадебный", "свадьба": "Свадебный обряд", "слёзы": "Слезы",
    "труба": "Труба печная", "украшение": "Украшения", "урожай": "Урожай-неурожай",
    "устная  история": "Устная история", "храм": "Церковь", "чёрный цвет": "Черный цвет", "яйца": "Яйцо"}

In [58]:
data = []
for idx, words in df[['id', 'ключевые слова']].values:
    idx_db = texts[idx]
    if type(words) == str:
        words = words.lower()
        if 'яйцо пасх. время' in words:
            words = words.replace('яйцо пасх. время', 'яйцо пасх., время')
        if ',' in words:
            sep = ','
        else:
            sep = '.'
        for word in words.strip().split(sep):
            if word:
                word = word.lower().strip('.').strip()
                if word in keywords:
                    data.append((idx_db, keywords[word]))
                elif word in keyword_mapping:
                    wrd = keyword_mapping[word].lower().strip('.').strip()
                    data.append((idx_db, keywords[wrd]))
                else:
                    print(word)

деревня
деревня
первый день
предписания
предписания
предписания
предписания
беседа


In [63]:
cur.execute("SELECT max(id) FROM t_k")
cur.fetchall()
#122618

[(124992,)]

In [62]:
cur.executemany("INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)", data)
db.commit()

**5. Собиратели**

In [70]:
# splitting, colelctors map
data = []
for idx, inform in df[['id', 'собиратели']].values:
    idx_db = texts[idx]
    if type(inform) != str:
        continue
    else:
        infs = splitting[inform]
        for i in infs:
            data.append((idx_db, collector_map[i]))

In [73]:
cur.executemany("INSERT INTO t_c (id_text, id_collector) VALUES (%s, %s)", data)
db.commit()

**6. Информанты**

In [114]:
def split_informants(text):
    if ',' in text:
        sep = ','
    elif ';' in text:
        sep = ';'
    elif '\n' in text:
        sep = '\n'
    else:
        sep = None
    if sep:
        infs = [i.strip() for i in text.split(sep)]
    else:
        infs = [text]
#     print(infs)
    return infs

In [92]:
import numpy as np

In [88]:
cur.executemany(
    """INSERT INTO informators (
    code, name, gender, birth_year, birth_village, current_village
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    """,
    table["informators"].replace(np.nan, None).values.tolist()
)
db.commit()

In [93]:
cur.execute("SELECT name, id FROM informators")
informators = {x: y for x, y in cur.fetchall()}

In [106]:
# df['информанты'].drop_duplicates()

In [115]:
data = []
for idx, text in df[['id', 'информанты']].values:
    idx_db = texts[idx]
    if type(text) != str:
        continue
    else:
#         print(text)
        infs = split_informants(text.strip())
#         print(infs)
        for i in infs:
            data.append((idx_db, informators[i]))

In [117]:
cur.executemany("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", data)
db.commit()